## Goal:
Build a basic chatbot that provides supportive and empathetic responses for stress, anxiety,
and emotional wellnes

“For this Task , I am fine-tunning the distilgpt2 model using the EmpatheticDialogues dataset. I use Hugging Face’s Trainer API to further train the model on emotionally supportive conversations. This helps the model respond kindly to mental health-related prompts.”

  ##  Objective:
      Loading the dataset

      Tokenizing and preparing

      Fine-tuning distilgpt2 using Hugging Face’s Trainer

      Testing the chatbot

    


In [ ]:
# intsalling dataset
# !pip install datasets transformers
!pip install --upgrade datasets transformers




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
# login to hugging face account

# from huggingface_hub import login
# login()

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer , AutoModelForCausalLM


## Load Dataset

In [ ]:
# Set a specific cache directory that Colab can handle
dataset = load_dataset("empathetic_dialogues", cache_dir="/content/hf_cache")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

The repository for empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

In [ ]:
# Check available splits
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})


In [ ]:
dataset.shape

{'train': (76673, 8), 'validation': (12030, 8), 'test': (10943, 8)}

In [ ]:
# Show a few examples
dataset['train'].shuffle(seed=42).select(range(5))


Dataset({
    features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
    num_rows: 5
})

    Needed Columns:
              prompt: Input to the bot
              utterance:	Output the bot should learn to generate

## Preprocess the dataset

In [ ]:
model_name = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# using prompt for user input and utterance for bot's output from dataset as a columns

def formate_dialogue(example):
    return{
      "text" :f"<|user|>{example['prompt']} <|bot|> {example['utterance']}   "
    }

#using a small subset

dataset=dataset['train'].select(range(4000)) # Keep it small to fit our system
formated_dataset=dataset.map(formate_dialogue)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
print(formated_dataset)

Dataset({
    features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags', 'text'],
    num_rows: 4000
})


In [ ]:
formated_dataset.select(range(200)) # checking its sample

Dataset({
    features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags', 'text'],
    num_rows: 200
})

## Tokenize and Prepare for Training

In [ ]:
# tokenizing dataset

tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(example):
    # Tokenize the text
    tokenized_example = tokenizer(example['text'], padding='max_length', truncation=True, max_length=128)
    # Add 'labels' key by copying 'input_ids'
    tokenized_example['labels'] = tokenized_example['input_ids']
    return tokenized_example

tokenized_dataset = formated_dataset.map(tokenize_function, batched= True)
tokenized_dataset.set_format('torch')

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

## Define Trainer and Train



In [ ]:
from transformers import Trainer , TrainingArguments
import os

os.environ["WANDB_DISABLED"] = "true"


training_args= TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=100,
    warmup_steps=10,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=False  # must be False on CPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
100,1.206600
200,1.113700
300,1.125600
400,1.076200
500,1.021000
600,1.004900
700,1.058700
800,1.075200
900,1.004600
1000,1.056500


TrainOutput(global_step=2000, training_loss=1.0212806358337403, metrics={'train_runtime': 7353.3283, 'train_samples_per_second': 0.544, 'train_steps_per_second': 0.272, 'total_flos': 130648375296000.0, 'train_loss': 1.0212806358337403, 'epoch': 1.0})

## test your chatbot

In [ ]:
def chat_with_bot(message):
    prompt = f"<|user|> {message} <|bot|>"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"], max_new_tokens=100, pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "<|bot|>" in response:
        return response.split("<|bot|>")[-1].strip()
    return response


In [ ]:
while True:
    msg = input("You: ")
    if msg.lower() in ["exit", "quit"]:
        break
    print("Bot:", chat_with_bot(msg))


You: i feel happy
Bot: I am happy to hear that.
You: are you sad
Bot: I am sad to hear that.
You: i am anxious
Bot: I am anxious to start a new job.
You: it feels wierd
Bot: I am so happy.
You: why are you happy?
Bot: I am happy to be a part of the world
You: i cannot sleep due to stress
Bot: I am so stressed.
You: i lost mt friend
Bot: i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend i lost my friend
You: i am sad
Bot: I am sad to hear that.
You: exit


This chatbot is build by distilgpt2 pretrained model which is used for small projects and mostly for practice purposes and may not give you concise and too much relevant response thats is why it is not giving me proper and expected out and that is the reason i am not converting this chatbot into streamlit or flask based webapp .On the other hand i could use mistral-7B-instruct pretrained model for this project but due to my system and having no GPU ,i ignored that because my system cannot handle large models.